# Tratamento dos dados do Twitter
Nesta etapa do trabalho será realizado o tratamento dos dados para identificação das palavras específicas
utilizadas nos tweets relacionados ao tema estudado. 
Para isso os passos abaixo serão necessários:
    1. Carregar o dataset com os tweets coletados
    2. Tratar os tweets fazendo o seguinte:
        - remover urls
        - remover usuarios e hashtags (também retirará os itens das consultas)
        - torná-las minuculas
        - remover acentos e pontuacao
    3. Identificar as palavras, removendo as stopwords


In [ ]:
!pip install unidecode

In [18]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import unidecode
import itertools
import collections

In [54]:
#lendo os tweeds gravados na etapa anterior e carregando em um data frame 
df = pd.read_pickle('dados_twitter_ans_classificados.pkl')
df.shape

(451, 16)

### Retirando os links dos tweets
A primeira limpeza a ser realizada é retirar links do tweet, para isso será utilizada a funçao abaixo.
Iremos adicionar uma nova coluna no dataframe com o tweet tratado.

In [55]:
def remove_url(txt):
    
    return " ".join(re.sub(r'http\S+', '', txt, flags=re.MULTILINE).split())

In [56]:
#adicionando a coluna text_tratada removendo as urls
df["text_tratado"] = df.text.apply(remove_url)
df[['text','text_tratado']]

,text,text_tratado
0,@djalmaneryneto @jornaldaclube @hapvidasaude P...,@djalmaneryneto @jornaldaclube @hapvidasaude P...
2,@samiabomfim @andy_balsan @hapvidasaude só dec...,@samiabomfim @andy_balsan @hapvidasaude só dec...
3,@hapvidasaude Quem apoia genocida nao esta do ...,@hapvidasaude Quem apoia genocida nao esta do ...
6,@samiabomfim @hapvidasaude já pode ser chamad...,@samiabomfim @hapvidasaude já pode ser chamada...
7,@samiabomfim @nemanema por isso que a @hapvida...,@samiabomfim @nemanema por isso que a @hapvida...
...,...,...
646,@alobradescoseg @bradescoseguros \nQueria muit...,@alobradescoseg @bradescoseguros Queria muito ...
647,"@pinheirotarsila @ANS_reguladora Társila, rece...","@pinheirotarsila @ANS_reguladora Társila, rece..."
648,"@acarneirinho Olá, boa tarde! Nosso perfil é o...","@acarneirinho Olá, boa tarde! Nosso perfil é o..."
649,Quais são as 10 profissões do futuro quando pe...,Quais são as 10 profissões do futuro quando pe...


### Retirando os usuários e hashtags
Removendo os usuários e hashtags que não serão necessários para identificação da consulta.
Esta etapa também retirará os itens de pesquisa utilizados na pesquisa.

In [57]:
def remove_users(txt):
    
    return " ".join(re.sub(r'@\S+',"",txt).split())

In [58]:
def remove_hashtags(txt):
    
    return " ".join(re.sub(r'#\S+',"",txt).split())

In [59]:
df["text_tratado"] = df.text_tratado.apply(remove_users)
df["text_tratado"] = df.text_tratado.apply(remove_hashtags)
df[['text','text_tratado']]

,text,text_tratado
0,@djalmaneryneto @jornaldaclube @hapvidasaude P...,Pra mim é um jornal imparcial que vale a pena ...
2,@samiabomfim @andy_balsan @hapvidasaude só dec...,só decepção
3,@hapvidasaude Quem apoia genocida nao esta do ...,"Quem apoia genocida nao esta do lado certo, nã..."
6,@samiabomfim @hapvidasaude já pode ser chamad...,já pode ser chamada de Hapmorte?
7,@samiabomfim @nemanema por isso que a @hapvida...,por isso que a é uma bosta de plano de saude.....
...,...,...
646,@alobradescoseg @bradescoseguros \nQueria muit...,Queria muito conseguir falar com uma pessoa qu...
647,"@pinheirotarsila @ANS_reguladora Társila, rece...","Társila, recebi sua mensagem e irei enviar a r..."
648,"@acarneirinho Olá, boa tarde! Nosso perfil é o...","Olá, boa tarde! Nosso perfil é o porém, orient..."
649,Quais são as 10 profissões do futuro quando pe...,Quais são as 10 profissões do futuro quando pe...


### Alterando o texto para lowercase
O objetivo é auxiliar na comparação do texto

In [60]:
df["text_tratado"] = df.text_tratado.apply(str.lower)
df[['text','text_tratado']]

,text,text_tratado
0,@djalmaneryneto @jornaldaclube @hapvidasaude P...,pra mim é um jornal imparcial que vale a pena ...
2,@samiabomfim @andy_balsan @hapvidasaude só dec...,só decepção
3,@hapvidasaude Quem apoia genocida nao esta do ...,"quem apoia genocida nao esta do lado certo, nã..."
6,@samiabomfim @hapvidasaude já pode ser chamad...,já pode ser chamada de hapmorte?
7,@samiabomfim @nemanema por isso que a @hapvida...,por isso que a é uma bosta de plano de saude.....
...,...,...
646,@alobradescoseg @bradescoseguros \nQueria muit...,queria muito conseguir falar com uma pessoa qu...
647,"@pinheirotarsila @ANS_reguladora Társila, rece...","társila, recebi sua mensagem e irei enviar a r..."
648,"@acarneirinho Olá, boa tarde! Nosso perfil é o...","olá, boa tarde! nosso perfil é o porém, orient..."
649,Quais são as 10 profissões do futuro quando pe...,quais são as 10 profissões do futuro quando pe...


### Remover acentos e pontuacao
Removendo os acentos e pontuações do texto

In [61]:
df["text_tratado"] = df['text_tratado'].str.replace('[^\w\s]','')
df["text_tratado"] = df['text_tratado'].apply(unidecode.unidecode)
df[['text','text_tratado']]

,text,text_tratado
0,@djalmaneryneto @jornaldaclube @hapvidasaude P...,pra mim e um jornal imparcial que vale a pena ...
2,@samiabomfim @andy_balsan @hapvidasaude só dec...,so decepcao
3,@hapvidasaude Quem apoia genocida nao esta do ...,quem apoia genocida nao esta do lado certo nao...
6,@samiabomfim @hapvidasaude já pode ser chamad...,ja pode ser chamada de hapmorte
7,@samiabomfim @nemanema por isso que a @hapvida...,por isso que a e uma bosta de plano de saude e...
...,...,...
646,@alobradescoseg @bradescoseguros \nQueria muit...,queria muito conseguir falar com uma pessoa qu...
647,"@pinheirotarsila @ANS_reguladora Társila, rece...",tarsila recebi sua mensagem e irei enviar a re...
648,"@acarneirinho Olá, boa tarde! Nosso perfil é o...",ola boa tarde nosso perfil e o porem orientamo...
649,Quais são as 10 profissões do futuro quando pe...,quais sao as 10 profissoes do futuro quando pe...


### Identificando as palavras frequentes
Nesta etapa serão removidas as stopwords identificando as palavras mais frequentes relacionadas ao tema.
StopWords são palavras muito frequentes e não discriminantes. São artigos, preposições, conjunções etc.


In [62]:
#separando o tweet tratado em palavras
df["text_tratado"] = df['text_tratado'].str.split()
df[['text','text_tratado']]

,text,text_tratado
0,@djalmaneryneto @jornaldaclube @hapvidasaude P...,"[pra, mim, e, um, jornal, imparcial, que, vale..."
2,@samiabomfim @andy_balsan @hapvidasaude só dec...,"[so, decepcao]"
3,@hapvidasaude Quem apoia genocida nao esta do ...,"[quem, apoia, genocida, nao, esta, do, lado, c..."
6,@samiabomfim @hapvidasaude já pode ser chamad...,"[ja, pode, ser, chamada, de, hapmorte]"
7,@samiabomfim @nemanema por isso que a @hapvida...,"[por, isso, que, a, e, uma, bosta, de, plano, ..."
...,...,...
646,@alobradescoseg @bradescoseguros \nQueria muit...,"[queria, muito, conseguir, falar, com, uma, pe..."
647,"@pinheirotarsila @ANS_reguladora Társila, rece...","[tarsila, recebi, sua, mensagem, e, irei, envi..."
648,"@acarneirinho Olá, boa tarde! Nosso perfil é o...","[ola, boa, tarde, nosso, perfil, e, o, porem, ..."
649,Quais são as 10 profissões do futuro quando pe...,"[quais, sao, as, 10, profissoes, do, futuro, q..."


In [63]:
#colocando todas as palavras em uma única lista
todas_palavras = list(itertools.chain(*df['text_tratado']))

#contando as palavras
frequencia_palavras = collections.Counter(todas_palavras)
frequencia_palavras.most_common(15)


[('de', 222),
 ('e', 220),
 ('o', 173),
 ('a', 164),
 ('que', 135),
 ('nao', 98),
 ('do', 77),
 ('da', 77),
 ('com', 67),
 ('em', 56),
 ('para', 55),
 ('uma', 51),
 ('um', 48),
 ('no', 46),
 ('saude', 42)]

In [49]:
#baixando as stopwords
nltk.download('stopwords')
stop_words_acento = set(stopwords.words('portuguese'))
#retirando acentuacao
stop_words = set([unidecode.unidecode(word) for word in stop_words_acento])
print(stop_words)

{'estas', 'tive', 'nossas', 'nossa', 'seremos', 'sera', 'estivera', 'essas', 'estiveramos', 'quando', 'houveramos', 'seja', 'houveriamos', 'hei', 'meu', 'eles', 'ja', 'aos', 'que', 'delas', 'tenham', 'estejamos', 'tivessemos', 'tem', 'tambem', 'seus', 'se', 'teria', 'sou', 'como', 'teremos', 'estive', 'tuas', 'aquele', 'em', 'estou', 'esses', 'dele', 'estiverem', 'houvemos', 'fomos', 'estao', 'isto', 'teriam', 'a', 'houve', 'teu', 'estavam', 'houvessem', 'mais', 'de', 'tinha', 'tivemos', 'houverem', 'seriam', 'eram', 'este', 'estavamos', 'houvermos', 'tiveramos', 'houvesse', 'ou', 'houvessemos', 'nem', 'havemos', 'seriamos', 'os', 'muito', 'serao', 'seu', 'dos', 'esteja', 'minha', 'pelos', 'temos', 'houverei', 'da', 'foramos', 'tera', 'nao', 'estamos', 'houvera', 'tiveram', 'tiverem', 'houverao', 'teriamos', 'suas', 'forem', 'por', 'pelas', 'estivemos', 'hajam', 'ele', 'fora', 'tivera', 'terei', 'sua', 'fossemos', 'isso', 'te', 'estiveram', 'tenha', 'mas', 'mesmo', 'fui', 'houveriam', 

[nltk_data] Downloading package stopwords to /Users/lups/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [64]:
#removendo as stop_words
for index,row in df.iterrows():
    for palavra in row['text_tratado']:
        if palavra in stop_words:
            df.at[index,'text_tratado'].remove(palavra)

todas_palavras = list(itertools.chain(*df['text_tratado']))            
frequencia_palavras = collections.Counter(todas_palavras)
frequencia_palavras.most_common(200)

[('o', 44),
 ('saude', 42),
 ('a', 39),
 ('e', 33),
 ('medico', 29),
 ('nao', 28),
 ('pra', 27),
 ('que', 26),
 ('plano', 22),
 ('um', 20),
 ('uma', 19),
 ('consulta', 19),
 ('dia', 18),
 ('atendimento', 17),
 ('ta', 15),
 ('dm', 14),
 ('bom', 14),
 ('vc', 14),
 ('ser', 13),
 ('vcs', 13),
 ('planos', 13),
 ('covid', 13),
 ('os', 13),
 ('ai', 13),
 ('q', 12),
 ('medicos', 12),
 ('hapvida', 12),
 ('ter', 11),
 ('eu', 11),
 ('vai', 11),
 ('agora', 11),
 ('so', 11),
 ('ola', 11),
 ('tata', 11),
 ('faz', 10),
 ('esta', 10),
 ('fazer', 10),
 ('de', 10),
 ('sei', 10),
 ('se', 10),
 ('no', 10),
 ('numero', 10),
 ('sulamerica', 10),
 ('pode', 9),
 ('vou', 9),
 ('pandemia', 9),
 ('empresarios', 9),
 ('meu', 9),
 ('me', 9),
 ('kit', 9),
 ('10', 9),
 ('com', 8),
 ('minha', 8),
 ('cloroquina', 8),
 ('empresa', 8),
 ('hospital', 8),
 ('essa', 8),
 ('melhor', 8),
 ('contra', 8),
 ('tem', 8),
 ('fiz', 8),
 ('aqui', 8),
 ('foi', 8),
 ('sobre', 8),
 ('ajuda', 8),
 ('hoje', 8),
 ('todo', 8),
 ('p', 7),
 

In [65]:
df_frequencia_palavras = pd.Series(frequencia_palavras)
df_frequencia_palavras.to_pickle('frequencia_palavras.pkl')
df.to_pickle('dados_twitter_ans_classificados_tratados.pkl')